In [1]:
import numpy as np

from skimage.io import imshow
from skimage.transform import resize

from sklearn.model_selection import train_test_split

from collections import namedtuple

import threading

from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam

from resize_generator import ResizeImageGenerator
%matplotlib inline

Using TensorFlow backend.


In [2]:
def get_train_data(path='public_data/00_input/train/'):
    from skimage.io import imread
    with open(path + 'gt.csv') as navigation_file:
        splits = [line.split(',') for line in navigation_file]
        points_number = int((len(splits[0]) - 1) / 2)
        train_x = np.array([imread(path + 'images/' + split[0]) for split in splits[1:]])
        train_y = np.array([[int(x) for x in split[1:]] for split in splits[1:]])
        return train_x, train_y, points_number

def get_colored_data(images, coordinates):
    return (
        np.array([image for image in images if len(np.shape(image)) == 3]),
        np.array([answer for image, answer in zip(images, coordinates) if len(np.shape(image)) == 3])
    )

In [3]:
all_x, all_y, points_number = get_train_data()
colored_x, colored_y = get_colored_data(all_x, all_y)
input_shape=(100, 100, 3)
train_x, test_x, train_y, test_y = train_test_split(
    colored_x, colored_y, test_size=0.3, random_state=42
)

In [4]:
regularization_lambda = 1e-2

def add_Conv2D_relu(model, n_filter, filters_size, input_shape=None):
    if input_shape is not None:
        model.add(Conv2D(
            n_filter, filters_size, padding='same',
            kernel_regularizer=regularizers.l2(regularization_lambda),
            activation='elu', input_shape=input_shape,
        ))
    else:
        model.add(Conv2D(
            n_filter, filters_size, padding='same',
            kernel_regularizer=regularizers.l2(regularization_lambda),
            activation='elu'
        ))

model = Sequential()

add_Conv2D_relu(model, 32, (3, 3), input_shape)
add_Conv2D_relu(model, 32, (3, 3))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

add_Conv2D_relu(model, 64, (3, 3))
add_Conv2D_relu(model, 64, (3, 3))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

add_Conv2D_relu(model, 128, (3, 3))
add_Conv2D_relu(model, 128, (3, 3))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Flatten(name='flatten'))
model.add(Dense(
    512, activation='elu',
    kernel_regularizer=regularizers.l2(regularization_lambda)
))
model.add(Dropout(0.5))
model.add(Dense(
    512, activation='elu',
    kernel_regularizer=regularizers.l2(regularization_lambda)
))

model.add(Dense(points_number * 2))

In [5]:
generator = ResizeImageGenerator(input_shape)

In [6]:
model.compile(loss='mean_squared_error',
              optimizer=Adam(decay=1e-3),
              metrics=['mse'])

In [7]:
class MyClallback:
    def __init__(self, model, directory):
        self._model = model
        self._directory = directory
        self._counter = 0

    def set_model(self, model):
        pass

    def set_params(self, params):
        pass

    def on_train_begin(self, logs):
        pass

    def on_train_end(self, logs):
        pass

    def on_epoch_begin(self, epoch, logs):
        pass

    def on_epoch_end(self, epoch, logs):
        self._model.save(
            self._directory + '/' + str(self._counter) +
            '_mse:_' + str(logs['mean_squared_error']) +
            '_val_mse:_' + str(logs['val_mean_squared_error']) +
            '.npz'
        )
        self._counter += 1

    def on_batch_begin(self, batch, logs):
        pass

    def on_batch_end(self, batch, logs):
        pass

In [8]:
batch_size = 32
model.fit_generator(
    generator.flow(train_x, train_y, batch_size=batch_size),
    steps_per_epoch=int(len(train_y) / batch_size), epochs=1000,
    callbacks=[MyClallback(model, 'small_models')],
    validation_data=generator.flow(test_x, test_y, batch_size=batch_size),
    validation_steps=int(len(test_y) / batch_size)
)

Epoch 1/1000
130/130 [==============================] - 28s - loss: 147.8613 - mean_squared_error: 127.1226 - val_loss: 646.0297 - val_mean_squared_error: 625.6604
Epoch 2/1000
130/130 [==============================] - 24s - loss: 56.0387 - mean_squared_error: 36.2717 - val_loss: 110.2442 - val_mean_squared_error: 91.3288
Epoch 3/1000
130/130 [==============================] - 24s - loss: 44.5165 - mean_squared_error: 26.4113 - val_loss: 43.8373 - val_mean_squared_error: 26.5042
Epoch 4/1000
130/130 [==============================] - 25s - loss: 36.2596 - mean_squared_error: 19.5596 - val_loss: 33.9502 - val_mean_squared_error: 17.8126
Epoch 5/1000
130/130 [==============================] - 25s - loss: 33.1635 - mean_squared_error: 17.5741 - val_loss: 27.4891 - val_mean_squared_error: 12.4034
Epoch 6/1000
130/130 [==============================] - 24s - loss: 30.5682 - mean_squared_error: 15.9597 - val_loss: 31.1649 - val_mean_squared_error: 16.9237
Epoch 7/1000
130/130 [=============

130/130 [==============================] - 24s - loss: 9.1143 - mean_squared_error: 4.8337 - val_loss: 13.5561 - val_mean_squared_error: 9.3187
Epoch 53/1000
130/130 [==============================] - 25s - loss: 8.9356 - mean_squared_error: 4.7382 - val_loss: 15.4389 - val_mean_squared_error: 11.2715
Epoch 54/1000
130/130 [==============================] - 24s - loss: 8.6339 - mean_squared_error: 4.4905 - val_loss: 13.7848 - val_mean_squared_error: 9.5938
Epoch 55/1000
130/130 [==============================] - 25s - loss: 8.9821 - mean_squared_error: 4.8314 - val_loss: 13.6471 - val_mean_squared_error: 9.5404
Epoch 56/1000
130/130 [==============================] - 25s - loss: 8.5848 - mean_squared_error: 4.5191 - val_loss: 12.9423 - val_mean_squared_error: 8.9243
Epoch 57/1000
130/130 [==============================] - 25s - loss: 8.3790 - mean_squared_error: 4.4006 - val_loss: 14.6002 - val_mean_squared_error: 10.6531
Epoch 58/1000
130/130 [==============================] - 24s - l

130/130 [==============================] - 24s - loss: 5.4738 - mean_squared_error: 3.0752 - val_loss: 9.8016 - val_mean_squared_error: 7.4097
Epoch 104/1000
130/130 [==============================] - 24s - loss: 5.3444 - mean_squared_error: 2.9668 - val_loss: 12.2881 - val_mean_squared_error: 9.9264
Epoch 105/1000
130/130 [==============================] - 26s - loss: 5.1788 - mean_squared_error: 2.8271 - val_loss: 11.7408 - val_mean_squared_error: 9.3951
Epoch 106/1000
130/130 [==============================] - 25s - loss: 5.1999 - mean_squared_error: 2.8641 - val_loss: 12.6602 - val_mean_squared_error: 10.3154
Epoch 107/1000
130/130 [==============================] - 25s - loss: 5.1085 - mean_squared_error: 2.7774 - val_loss: 15.5913 - val_mean_squared_error: 13.2690
Epoch 108/1000
130/130 [==============================] - 24s - loss: 5.1001 - mean_squared_error: 2.7933 - val_loss: 11.4997 - val_mean_squared_error: 9.2069
Epoch 109/1000
130/130 [==============================] - 24

130/130 [==============================] - 24s - loss: 4.0569 - mean_squared_error: 2.1972 - val_loss: 11.6574 - val_mean_squared_error: 9.8053
Epoch 155/1000
130/130 [==============================] - 24s - loss: 3.9826 - mean_squared_error: 2.1354 - val_loss: 11.7702 - val_mean_squared_error: 9.9263
Epoch 156/1000
130/130 [==============================] - 26s - loss: 3.9237 - mean_squared_error: 2.0837 - val_loss: 11.6347 - val_mean_squared_error: 9.8042
Epoch 157/1000
130/130 [==============================] - 26s - loss: 4.0321 - mean_squared_error: 2.1995 - val_loss: 10.6219 - val_mean_squared_error: 8.7924
Epoch 158/1000
130/130 [==============================] - 25s - loss: 3.9936 - mean_squared_error: 2.1704 - val_loss: 11.6691 - val_mean_squared_error: 9.8505
Epoch 159/1000
130/130 [==============================] - 25s - loss: 4.0003 - mean_squared_error: 2.1876 - val_loss: 12.3507 - val_mean_squared_error: 10.5429
Epoch 160/1000
130/130 [==============================] - 24

130/130 [==============================] - 25s - loss: 3.3718 - mean_squared_error: 1.8580 - val_loss: 10.0964 - val_mean_squared_error: 8.5781
Epoch 206/1000
 25/130 [====>.........................] - ETA: 11s - loss: 3.3333 - mean_squared_error: 1.8161

KeyboardInterrupt: 